In [1]:
!pip install gekko

In [2]:
from gekko import GEKKO
import numpy as np
import random
from baconshor import *
import sys
import os
from contextlib import contextmanager

In [9]:
#surpress the print statement from the solver

@contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout  # Backup current stdout
        sys.stdout = devnull  # Redirect stdout to devnull (suppress)
        try:
            yield
        finally:
            sys.stdout = old_stdout  # Restore original stdout
# settings ----
d=3
num_qubits =d**2
num_stabilizers =(d - 1)**2


ps = {i: p for i in range(num_qubits)}

grid = random_error_grid(d,p)
print("ErrorGrid: ")
Print(grid)

Cs = construct_stabilizers(d,grid)
# print("Stabilizer Measurements: ")
# print(Cs)

# set up gekko ----
m = GEKKO(remote=False)
m.options.SOLVER = 1  # APOPT is an MINLP solver
# optional solver settings with APOPT
m.solver_options = [
    "minlp_maximum_iterations 500",  # minlp iterations with integer solution
    "minlp_max_iter_with_int_sol 10",  # treat minlp as nlp
    "minlp_as_nlp 0",  
    # nlp sub-problem max iterations
    "nlp_maximum_iterations 50",  # 1 = depth first, 2 = breadth first
    "minlp_branch_method 1",  # maximum deviation from whole number
    "minlp_integer_tol 0.05",  # covergence tolerance
    "minlp_gap_tol 0.01",
    "print_level 0",
]


# set up variables ---
Es = {}
for i in range(num_qubits):
    Es[i] = m.Var(
        value=random.randint(0,1), lb=0, ub=1, integer=True
    )

Ks = {}
for k in Cs.keys():
    Ks[k] = m.Var(
        value=random.randint(0,1), lb=0, integer=True
    )


# Objective ---
m.Obj(
    -m.sum(
        [
            np.log(ps[j]) * Es[j]  + np.log(1 - ps[j]) * (1 - Es[j])
            for j in range(num_qubits)
        ]
    )
)


# Constraints ---

for key, val in Cs.items():
    i, j,k,l,o,n = key
    # m.Equation(Es[i] + Es[j] - 2 * Ks[key] == (1 - val)/2)
    m.Equation(m.sum([Es[i] for i in key]) - 2 * Ks[key] == (1 - val)/2) #automatically sums over all the qubits in the stabilizer key

with suppress_stdout():
    m.solve()

print("Solver's Guess: ")
Print_Solver(d, Es)
print("Was the solver correct? ")
print(solver_accuracy(d, grid,Es))


ErrorGrid: 
0 0 0 
0 0 0 
1 0 0 
Solver's Guess: 
0 1 0 
0 0 1 
0 0 0 
Was the solver correct? 
False


In [ ]:
#surpress the print statement from the solver

@contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout  # Backup current stdout
        sys.stdout = devnull  # Redirect stdout to devnull (suppress)
        try:
            yield
        finally:
            sys.stdout = old_stdout  # Restore original stdout
# settings ----
d=3
num_qubits =d**2
num_stabilizers =(d - 1)**2


physical_error_probs = np.linspace(0.01, 0.70, num=10)#physical error probs
logical_error_probs=[]
for p in physical_error_probs: 
    count = 0 #number of correct guesses from solver
    for i in range(100):

        ps = {i: p for i in range(num_qubits)}

        grid = random_error_grid(d,p)
        # print("ErrorGrid: ") 
        # Print(grid)

        Cs = construct_stabilizers(d,grid)
        # print("Stabilizer Measurements: ")
        # print(Cs)

        # set up gekko ----
        m = GEKKO(remote=False)
        m.options.SOLVER = 1  # APOPT is an MINLP solver
        # optional solver settings with APOPT
        m.solver_options = [
            "minlp_maximum_iterations 500",  # minlp iterations with integer solution
            "minlp_max_iter_with_int_sol 10",  # treat minlp as nlp
            "minlp_as_nlp 0",  
            # nlp sub-problem max iterations
            "nlp_maximum_iterations 50",  # 1 = depth first, 2 = breadth first
            "minlp_branch_method 1",  # maximum deviation from whole number
            "minlp_integer_tol 0.05",  # covergence tolerance
            "minlp_gap_tol 0.01",
            "print_level 0",
        ]


        # set up variables ---
        Es = {}
        for i in range(num_qubits):
            Es[i] = m.Var(
                value=random.randint(0,1), lb=0, ub=1, integer=True
            )

        Ks = {}
        for k in Cs.keys():
            Ks[k] = m.Var(
                value=random.randint(0,1), lb=0, integer=True
            )


        # Objective ---
        m.Obj(
            -m.sum(
                [
                    np.log(ps[j]) * Es[j]  + np.log(1 - ps[j]) * (1 - Es[j])
                    for j in range(num_qubits)
                ]
            )
        )


        # Constraints ---

        for key, val in Cs.items():
            i, j,k,l,o,n = key
            # m.Equation(Es[i] + Es[j] - 2 * Ks[key] == (1 - val)/2)
            m.Equation(m.sum([Es[i] for i in key]) - 2 * Ks[key] == (1 - val)/2) #automatically sums over all the qubits in the stabilizer key

        with suppress_stdout():
            m.solve()

        # print("Solver's Guess: ")
        # Print_Solver(d, Es)
        # print("Was the solver correct? ")
        # print(solver_accuracy(d, grid,Es))
        if(solver_accuracy):
            count+=1
    logical_error_probs.append(count/1024)

    


In [8]:
#make plot p vs percent decoder was right - run each p like 1000 times 
print(logical_error_probs)

[0.09765625, 0.09765625, 0.09765625, 0.09765625, 0.09765625, 0.09765625, 0.09765625, 0.09765625, 0.09765625, 0.09765625]


In [ ]:
#mle for rep code, graph
